In [1]:
import pandas as pd
master_df = pd.read_excel('master.xlsx')
slave_df = pd.read_excel('slave.xlsx')
print('Master excel file')
master_df.head()

Master excel file


,chapter_number,chapter_name,section,icd_codes,Unnamed: 4
0,Chapter 1,Certain infectious and parasitic diseases (A00...,NaN,NaN,NaN
1,NaN,NaN,A00-A09 Intestinal infectious diseases,NaN,NaN
2,NaN,NaN,A15-A19 Tuberculosis,NaN,NaN
3,NaN,NaN,A20-A28 Certain zoonotic bacterial diseases,NaN,NaN
4,NaN,NaN,A30-A49 Other bacterial diseases,NaN,NaN


In [2]:
chapter_parent_df = master_df[(master_df.chapter_number.notnull())]
chapter_parent_df_indices = list(chapter_parent_df.index)
chapter_parent_df_indices

[0,
 23,
 45,
 53,
 64,
 76,
 88,
 101,
 107,
 118,
 130,
 141,
 151,
 172,
 184,
 194,
 207,
 219,
 234,
 256,
 291,
 307]

In [3]:
def get_all_codes(start_index, end_index):
    label = master_df[start_index + 1 : end_index].section
    all_codes = label.map(lambda str: [str, str.split(' ')[0].split('-')])
    return list(all_codes)

In [4]:
rows_to_insert = []
for index, element in enumerate(chapter_parent_df_indices):
    start_index = chapter_parent_df_indices[index]
    chapter_name = master_df.iloc[start_index].chapter_name
    chapter_number = master_df.iloc[start_index].chapter_number
    if index == len(chapter_parent_df_indices) - 1:
        end_index = index
    else:
        end_index = chapter_parent_df_indices[index + 1]
    sliced_df = master_df.iloc[start_index : end_index]
    dict = {'chapter_name': chapter_name,
            'chapter_number': chapter_number,
            'icd_codes': [],
            'chapter_codes': get_all_codes(start_index, end_index)}
    
    rows_to_insert.append(dict)

In [5]:
rows_to_insert

[{'chapter_name': 'Certain infectious and parasitic diseases (A00-B99)',
  'chapter_number': 'Chapter 1',
  'icd_codes': [],
  'chapter_codes': [['A00-A09 Intestinal infectious diseases', ['A00', 'A09']],
   ['A15-A19 Tuberculosis', ['A15', 'A19']],
   ['A20-A28 Certain zoonotic bacterial diseases', ['A20', 'A28']],
   ['A30-A49 Other bacterial diseases', ['A30', 'A49']],
   ['A50-A64 Infections with a predominantly sexual mode of transmission',
    ['A50', 'A64']],
   ['A65-A69 Other spirochetal diseases', ['A65', 'A69']],
   ['A70-A74 Other diseases caused by chlamydiae', ['A70', 'A74']],
   ['A75-A79 Rickettsioses', ['A75', 'A79']],
   ['A80-A89 Viral and prion infections of the central nervous system',
    ['A80', 'A89']],
   ['A90-A99 Arthropod-borne viral fevers and viral hemorrhagic fevers',
    ['A90', 'A99']],
   ['B00-B09 Viral infections characterized by skin and mucous membrane lesions',
    ['B00', 'B09']],
   ['B10 Other human herpesviruses', ['B10']],
   ['B15-B19 Viral 

In [6]:
slave_df.head()

,code,level,short_description,long_description
0,A00,0,Cholera,Cholera
1,A000,1,"Cholera due to Vibrio cholerae 01, biovar chol...","Cholera due to Vibrio cholerae 01, biovar chol..."
2,A001,1,"Cholera due to Vibrio cholerae 01, biovar eltor","Cholera due to Vibrio cholerae 01, biovar eltor"
3,A009,1,"Cholera, unspecified","Cholera, unspecified"
4,A01,0,Typhoid and paratyphoid fevers,Typhoid and paratyphoid fevers


In [7]:
def find_parent_codes_between(first_code, last_code=''):
    if not last_code:
        return [first_code]
    sliced_df = slave_df[slave_df.code.between(first_code, last_code)]
    return list(sliced_df[sliced_df.code.str.len() == len(first_code)].code)

In [8]:
find_parent_codes_between('A00','A09')

['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09']

In [9]:
slave_df[slave_df.code.str.contains(f"^A010")]
#slave_df[slave_df.code.str.contains(f"^A010")].iloc[:1].code.str.len() == 4

,code,level,short_description,long_description
5,A010,0,Typhoid fever,Typhoid fever
6,A0100,1,"Typhoid fever, unspecified","Typhoid fever, unspecified"
7,A0101,1,Typhoid meningitis,Typhoid meningitis
8,A0102,1,Typhoid fever with heart involvement,Typhoid fever with heart involvement
9,A0103,1,Typhoid pneumonia,Typhoid pneumonia
10,A0104,1,Typhoid arthritis,Typhoid arthritis
11,A0105,1,Typhoid osteomyelitis,Typhoid osteomyelitis
12,A0109,1,Typhoid fever with other complications,Typhoid fever with other complications


In [10]:
# Chapter has many list of codes
def build_children(codes):
    if len(codes) == 1:
        codes.append('')
    print(codes)
    first_code, last_code = codes
    all_icd_codes = find_parent_codes_between(first_code, last_code)
    icd_list = []
    for code in all_icd_codes:
        icd_df = slave_df[slave_df.code.str.contains(f"^{code}")]
        parent_df = icd_df.iloc[0]
        parent_code_length = len(parent_df.code)
        parent = {
                    'code': parent_df.code,
                    'label': f"{parent_df.code} - {parent_df.long_description}",
                    'level': str(parent_df.level),
                    'children': []
                  }
        for _, row in icd_df[1:].iterrows():
            if row['level'] == 0:
                grand_children = {
                                    'code': row.code,
                                    'label': f"{row.code} - {row.long_description}",
                                    'level': str(row.level),
                                    'children': []
                                }
                grand_child_rows = slave_df[slave_df.code.str.contains(f"^{row.code}")]
                for _, child_row in grand_child_rows[1:].iterrows():
                    grand_children['children'].append({
                                            'code': child_row.code,
                                            'label': f"{child_row.code} - {child_row.long_description}",
                                            'level': str(child_row.level)
                    })
                parent['children'].append(grand_children)
                
            else:
                if parent_code_length != len(row.code) - 1:
                    continue
                parent['children'].append({
                                            'code': row.code,
                                            'label': f"{row.code} - {row.long_description}",
                                            'level': str(row.level)
                                          })
        icd_list.append(parent)
    #print(icd_list)
    return icd_list

In [11]:
# ['A00-A09 Intestinal infectious diseases', ['A00', 'A09']
def get_icd_codes(chapter_codes):
    chapter_list = []
    for item in chapter_codes:
        dict = {
                'code': item[-1],
                'label': item[0],
                'children': build_children(item[-1])
               }
        chapter_list.append(dict)
    return chapter_list

In [12]:
type(rows_to_insert)

list

In [13]:
for chapter in rows_to_insert:
    print(chapter)
    chapter['icd_codes'] = get_icd_codes(chapter['chapter_codes'])

{'chapter_name': 'Certain infectious and parasitic diseases (A00-B99)', 'chapter_number': 'Chapter 1', 'icd_codes': [], 'chapter_codes': [['A00-A09 Intestinal infectious diseases', ['A00', 'A09']], ['A15-A19 Tuberculosis', ['A15', 'A19']], ['A20-A28 Certain zoonotic bacterial diseases', ['A20', 'A28']], ['A30-A49 Other bacterial diseases', ['A30', 'A49']], ['A50-A64 Infections with a predominantly sexual mode of transmission', ['A50', 'A64']], ['A65-A69 Other spirochetal diseases', ['A65', 'A69']], ['A70-A74 Other diseases caused by chlamydiae', ['A70', 'A74']], ['A75-A79 Rickettsioses', ['A75', 'A79']], ['A80-A89 Viral and prion infections of the central nervous system', ['A80', 'A89']], ['A90-A99 Arthropod-borne viral fevers and viral hemorrhagic fevers', ['A90', 'A99']], ['B00-B09 Viral infections characterized by skin and mucous membrane lesions', ['B00', 'B09']], ['B10 Other human herpesviruses', ['B10']], ['B15-B19 Viral hepatitis', ['B15', 'B19']], ['B20 Human immunodeficiency v

In [14]:
# First level:- Pick chapter_name
rows_to_insert[0]

{'chapter_name': 'Certain infectious and parasitic diseases (A00-B99)',
 'chapter_number': 'Chapter 1',
 'icd_codes': [{'code': ['A00', 'A09'],
   'label': 'A00-A09 Intestinal infectious diseases',
   'children': [{'code': 'A00',
     'label': 'A00 - Cholera',
     'level': '0',
     'children': [{'code': 'A000',
       'label': 'A000 - Cholera due to Vibrio cholerae 01, biovar cholerae',
       'level': '1'},
      {'code': 'A001',
       'label': 'A001 - Cholera due to Vibrio cholerae 01, biovar eltor',
       'level': '1'},
      {'code': 'A009', 'label': 'A009 - Cholera, unspecified', 'level': '1'}]},
    {'code': 'A01',
     'label': 'A01 - Typhoid and paratyphoid fevers',
     'level': '0',
     'children': [{'code': 'A010',
       'label': 'A010 - Typhoid fever',
       'level': '0',
       'children': [{'code': 'A0100',
         'label': 'A0100 - Typhoid fever, unspecified',
         'level': '1'},
        {'code': 'A0101', 'label': 'A0101 - Typhoid meningitis', 'level': '1'},


In [15]:
# 2nd level: Pick code and label
# 3rd level: Pick code and label from children
len(rows_to_insert[0]['icd_codes'])

22

In [16]:
# If at 3rd level, more children are there, then at 4th level pick code and label from nested children list.
rows_to_insert[0]['icd_codes'][0]['children'][1]['children']

[{'code': 'A010',
  'label': 'A010 - Typhoid fever',
  'level': '0',
  'children': [{'code': 'A0100',
    'label': 'A0100 - Typhoid fever, unspecified',
    'level': '1'},
   {'code': 'A0101', 'label': 'A0101 - Typhoid meningitis', 'level': '1'},
   {'code': 'A0102',
    'label': 'A0102 - Typhoid fever with heart involvement',
    'level': '1'},
   {'code': 'A0103', 'label': 'A0103 - Typhoid pneumonia', 'level': '1'},
   {'code': 'A0104', 'label': 'A0104 - Typhoid arthritis', 'level': '1'},
   {'code': 'A0105', 'label': 'A0105 - Typhoid osteomyelitis', 'level': '1'},
   {'code': 'A0109',
    'label': 'A0109 - Typhoid fever with other complications',
    'level': '1'}]},
 {'code': 'A011', 'label': 'A011 - Paratyphoid fever A', 'level': '1'},
 {'code': 'A012', 'label': 'A012 - Paratyphoid fever B', 'level': '1'},
 {'code': 'A013', 'label': 'A013 - Paratyphoid fever C', 'level': '1'},
 {'code': 'A014',
  'label': 'A014 - Paratyphoid fever, unspecified',
  'level': '1'}]

In [17]:
# len(rows_to_insert[0]['icd_codes'])

In [18]:
# import json
# with open('real_json.json', 'w') as fout:
#     json.dump(rows_to_insert , fout)

In [19]:
# Since we know, there can only be 2 levels of children, skipped writing a recursive algorithm.
flat_data = []
for row in rows_to_insert:
    for code in row['icd_codes']:
        for child in code['children']:
            for index, grand_child in enumerate(child['children']):
                data = {
                        'chapter_name': row['chapter_name'],
                        'chapter_number': row['chapter_number'],
                        'parent_code': child['code'],
                        'parent_label': child['label'],
                        'parent_level': child['level']
                    }
                data['child_code'] = grand_child['code']
                data['child_label'] = grand_child['label']
                data['child_level'] = grand_child['level']
                if(grand_child['level'] == '0'):
                    for super_grand_child in grand_child['children']:
                        new_dict = {
                            'sub_child_code': super_grand_child['code'],
                            'sub_child_label': super_grand_child['label'],
                            'sub_child_level': super_grand_child['level']
                        }
                        dict = {**data, **new_dict}
                        flat_data.append(dict)
                else:
                    flat_data.append(data)

In [20]:
flat_data[4]

{'chapter_name': 'Certain infectious and parasitic diseases (A00-B99)',
 'chapter_number': 'Chapter 1',
 'parent_code': 'A01',
 'parent_label': 'A01 - Typhoid and paratyphoid fevers',
 'parent_level': '0',
 'child_code': 'A010',
 'child_label': 'A010 - Typhoid fever',
 'child_level': '0',
 'sub_child_code': 'A0101',
 'sub_child_label': 'A0101 - Typhoid meningitis',
 'sub_child_level': '1'}

In [21]:
# chapter_1_df = new_df[new_df.chapter_number == 'Chapter 1']
# chapter_1_df

In [22]:
#from json_normalize import json_normalize
new_df = pd.json_normalize(flat_data)

In [23]:
new_df

,chapter_name,chapter_number,parent_code,parent_label,parent_level,child_code,child_label,child_level,sub_child_code,sub_child_label,sub_child_level
0,Certain infectious and parasitic diseases (A00...,Chapter 1,A00,A00 - Cholera,0,A000,"A000 - Cholera due to Vibrio cholerae 01, biov...",1,NaN,NaN,NaN
1,Certain infectious and parasitic diseases (A00...,Chapter 1,A00,A00 - Cholera,0,A001,"A001 - Cholera due to Vibrio cholerae 01, biov...",1,NaN,NaN,NaN
2,Certain infectious and parasitic diseases (A00...,Chapter 1,A00,A00 - Cholera,0,A009,"A009 - Cholera, unspecified",1,NaN,NaN,NaN
3,Certain infectious and parasitic diseases (A00...,Chapter 1,A01,A01 - Typhoid and paratyphoid fevers,0,A010,A010 - Typhoid fever,0,A0100,"A0100 - Typhoid fever, unspecified",1
4,Certain infectious and parasitic diseases (A00...,Chapter 1,A01,A01 - Typhoid and paratyphoid fevers,0,A010,A010 - Typhoid fever,0,A0101,A0101 - Typhoid meningitis,1
...,...,...,...,...,...,...,...,...,...,...,...
211707,Factors influencing health status and contact ...,Chapter 21,Z99,Z99 - Dependence on enabling machines and devi...,0,Z991,Z991 - Dependence on respirator,0,Z9912,Z9912 - Encounter for respirator [ventilator] ...,1
211708,Factors influencing health status and contact ...,Chapter 21,Z99,Z99 - Dependence on enabling machines and devi...,0,Z992,Z992 - Dependence on renal dialysis,1,NaN,NaN,NaN
211709,Factors influencing health status and contact ...,Chapter 21,Z99,Z99 - Dependence on enabling machines and devi...,0,Z993,Z993 - Dependence on wheelchair,1,NaN,NaN,NaN
211710,Factors influencing health status and contact ...,Chapter 21,Z99,Z99 - Dependence on enabling machines and devi...,0,Z998,Z998 - Dependence on other enabling machines a...,0,Z9981,Z9981 - Dependence on supplemental oxygen,1


In [24]:
new_df.to_csv(r'/Users/nakumar/Desktop/icd/new_flat_data.csv', index=False)

In [25]:
# chapter_1_df = new_df[new_df.chapter_number == 'Chapter 1']
# chapter_1_df

In [26]:
#chapter_1_df.to_csv(r'/Users/nakumar/Desktop/icd/chapter_1_data.csv', index=False)